# Setup

In [159]:
import geopandas as gpd
import pandas as pd
import mapclassify
from matplotlib import pyplot as plt
import requests
import streamlit as st

# Read in data

In [160]:
def read_api(url):
    # Requests to pull the data
    response = requests.get(url)
    # If the request is accepted then...
    print(response.status_code)
    if response.status_code == 200:
        # Read in the data as a json file
        data = response.json()
        # Convert the json file into a GeoDataFrame (dataframe containing geometry)
        return gpd.GeoDataFrame(data)
    else:
        return None

# Read in permit data of completed new-build projects
newbuild_df = read_api("https://data.cityofchicago.org/resource/ydr8-5enu.json?permit_status=COMPLETE&permit_type=PERMIT - NEW CONSTRUCTION&$limit=50000")
# Read in permit data of completed retrofit projects
retrofit_df = read_api("https://data.cityofchicago.org/resource/ydr8-5enu.json?permit_status=COMPLETE&permit_type=PERMIT - RENOVATION/ALTERATION&$limit=50000")
# Read in energy benchmark data of all buildings
energy_df = read_api("https://data.cityofchicago.org/resource/xq83-jr8c.json?$limit=50000")

200
200
200


# Data cleaning

In [161]:
necessary_columns = [x for x in retrofit_df.columns if x.startswith('contact') == False]

retrofit_df = retrofit_df[necessary_columns]
newbuild_df = newbuild_df[necessary_columns]

In [162]:
display(energy_df.head())
print(energy_df.groupby('reporting_status')['reporting_status'].count())
print(energy_df.groupby('community_area')['community_area'].count().sort_values())
print(display(energy_df.groupby('data_year')['data_year'].count()))

,data_year,id,reporting_status,address,zip_code,chicago_energy_rating,exempt_from_chicago_energy_rating,community_area,gross_floor_area_buildings_sq_ft,latitude,...,district_steam_use_kbtu,district_chilled_water_use_kbtu,site_eui_kbtu_sq_ft,source_eui_kbtu_sq_ft,weather_normalized_site_eui_kbtu_sq_ft,weather_normalized_source_eui_kbtu_sq_ft,total_ghg_emissions_metric_tons_co2e,ghg_intensity_kg_co2e_sq_ft,water_use_kgal,all_other_fuel_use_kbtu
0,2021,101671,Not Submitted,6101 6115 N SEELEY AVE,60659,0,False,WEST RIDGE,65250,41.99046172,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021,101826,Not Submitted,8136 8142 S DREXEL AVE,60619,0,False,CHATHAM,66888,41.74401302,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021,102323,Not Submitted,3515 3525 W FRANKLIN BLVD,60624,0,False,HUMBOLDT PARK,93000,41.8808242,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022,256888,Submitted Data,318 N CARPENTER ST,60607,4,False,NaN,104849,41.88736471232649,...,0,0,32.4,61.9,32.8,61.4,311.5,3,NaN,NaN
4,2021,104374,Not Submitted,6000 N SHERIDAN RD,60660,0,False,EDGEWATER,55686,41.99090057,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


reporting_status
Exempt                957
Not Covered 2024      312
Not Submitted        3867
Submitted           17572
Submitted Data       5621
Name: reporting_status, dtype: int64
community_area
outside Chicago, in Bedford Park       1
Edison Park                            2
Burnside                               2
Beverly                                2
Lakeview                               3
                                    ... 
LINCOLN PARK                         990
LAKE VIEW                           1274
NEAR WEST SIDE                      1821
LOOP                                2670
NEAR NORTH SIDE                     3746
Name: community_area, Length: 157, dtype: int64


data_year
2014     243
2015    1521
2016    2717
2017    2797
2018    3430
2019    3438
2020    3582
2021    3550
2022    3613
2023    3438
Name: data_year, dtype: int64

None


In [163]:
newbuild_gdf = gpd.GeoDataFrame(newbuild_df)
retrofit_gdf = gpd.GeoDataFrame(retrofit_df)
energy_gdf = gpd.GeoDataFrame(energy_df)

# Dataviz

In [164]:
# Optional: Filter by year
#selected_year = st.slider("Select Year", int(energy_gdf['data_year'].min()), int(energy_gdf['data_year'].max()), step=1)
selected_year = 2023
df_map = energy_gdf[energy_gdf['data_year'] == selected_year]

st.map(df_map[['latitude', 'longitude']])

# Optional: Add extra info
st.write("Showing buildings for:", selected_year)
st.dataframe(df_map[['address', 'community_area']])

2025-04-10 16:27:52.997 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-10 16:27:52.997 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-10 16:27:52.999 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-10 16:27:53.000 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-10 16:27:53.001 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-10 16:27:53.001 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-10 16:27:54.608 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-10 16:27:54.608 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()

In [169]:
energy_df.columns

Index(['data_year', 'id', 'reporting_status', 'address', 'zip_code',
       'chicago_energy_rating', 'exempt_from_chicago_energy_rating',
       'community_area', 'gross_floor_area_buildings_sq_ft', 'latitude',
       'longitude', 'location', 'row_id', ':@computed_region_43wa_7qmu',
       ':@computed_region_vrxf_vc4k', ':@computed_region_6mkv_f3dw',
       ':@computed_region_bdys_3d7i', ':@computed_region_awaf_s7ux',
       'property_name', 'primary_property_type', 'year_built', 'of_buildings',
       'energy_star_score', 'electricity_use_kbtu', 'natural_gas_use_kbtu',
       'district_steam_use_kbtu', 'district_chilled_water_use_kbtu',
       'site_eui_kbtu_sq_ft', 'source_eui_kbtu_sq_ft',
       'weather_normalized_site_eui_kbtu_sq_ft',
       'weather_normalized_source_eui_kbtu_sq_ft',
       'total_ghg_emissions_metric_tons_co2e', 'ghg_intensity_kg_co2e_sq_ft',
       'water_use_kgal', 'all_other_fuel_use_kbtu'],
      dtype='object')

In [170]:
energy_df[['data_year','id','address','chicago_energy_rating','longitude','latitude','energy_star_score','electricity_use_kbtu','site_eui_kbtu_sq_ft']].head()

,data_year,id,address,chicago_energy_rating,longitude,latitude,energy_star_score,electricity_use_kbtu,site_eui_kbtu_sq_ft
0,2021,101671,6101 6115 N SEELEY AVE,0,-87.70620782,41.99046172,NaN,NaN,NaN
1,2021,101826,8136 8142 S DREXEL AVE,0,-87.60549928,41.74401302,NaN,NaN,NaN
2,2021,102323,3515 3525 W FRANKLIN BLVD,0,-87.72327934,41.8808242,NaN,NaN,NaN
3,2022,256888,318 N CARPENTER ST,4,-87.653505317143,41.88736471232649,98,1669219.6,32.4
4,2021,104374,6000 N SHERIDAN RD,0,-87.66660744,41.99090057,NaN,NaN,NaN


In [175]:
energy_df[['data_year','id','address','chicago_energy_rating','longitude','latitude','energy_star_score']].head()
energy_df_clean = energy_df.dropna(subset=['data_year', 'longitude', 'latitude'])
energy_df_clean

,data_year,id,reporting_status,address,zip_code,chicago_energy_rating,exempt_from_chicago_energy_rating,community_area,gross_floor_area_buildings_sq_ft,latitude,...,district_steam_use_kbtu,district_chilled_water_use_kbtu,site_eui_kbtu_sq_ft,source_eui_kbtu_sq_ft,weather_normalized_site_eui_kbtu_sq_ft,weather_normalized_source_eui_kbtu_sq_ft,total_ghg_emissions_metric_tons_co2e,ghg_intensity_kg_co2e_sq_ft,water_use_kgal,all_other_fuel_use_kbtu
0,2021,101671,Not Submitted,6101 6115 N SEELEY AVE,60659,0,False,WEST RIDGE,65250,41.99046172,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021,101826,Not Submitted,8136 8142 S DREXEL AVE,60619,0,False,CHATHAM,66888,41.74401302,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021,102323,Not Submitted,3515 3525 W FRANKLIN BLVD,60624,0,False,HUMBOLDT PARK,93000,41.8808242,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022,256888,Submitted Data,318 N CARPENTER ST,60607,4,False,NaN,104849,41.88736471232649,...,0,0,32.4,61.9,32.8,61.4,311.5,3,NaN,NaN
4,2021,104374,Not Submitted,6000 N SHERIDAN RD,60660,0,False,EDGEWATER,55686,41.99090057,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28324,2023,101753,Submitted,222 South Riverside Plaza,60606,4,NaN,Near West Side,1237455,41.874634,...,NaN,NaN,56.1,111.8,60.1,116,6936.2,5.6,19287.4,NaN
28325,2023,175987,Submitted,11840 S Marshfield Ave,60643,2.5,NaN,Morgan Park,129029,41.973128,...,NaN,NaN,64.1,126.8,65.6,131,820.2,6.4,NaN,NaN
28326,2023,250083,Submitted,1930 W. Loyola Ave,60626,2.5,NaN,West Ridge,99530,41.893483,...,NaN,NaN,105.4,151.3,117.5,163.2,757.9,7.6,NaN,NaN
28327,2023,256672,Submitted,2903-2923 N Claremont Ave,60618,3,NaN,Avondale,71737,41.901711,...,NaN,NaN,77.8,112.1,85.2,121.4,404.7,5.6,NaN,NaN


In [267]:
retrofit_df.columns
retrofit_df[['community_area','ward','street_number','street_name','census_tract']].head()

,community_area,ward,street_number,street_name,census_tract
0,27,24,739,INDEPENDENCE BLVD,270500
1,28,27,1628,WASHINGTON BLVD,280400
2,10,41,6414,NEW ENGLAND AVE,100200
3,8,42,321,CLARK ST,81700
4,9,41,7356,HARLEM AVE,90100


In [268]:
energy_df_clean = energy_df.dropna(subset=['data_year', 'longitude', 'latitude', 'energy_star_score'])
# Make columns numbers
energy_df_clean['data_year'] = pd.to_numeric(energy_df_clean['data_year'], errors='coerce')
energy_df_clean['longitude'] = pd.to_numeric(energy_df_clean['longitude'], errors='coerce')
energy_df_clean['latitude'] = pd.to_numeric(energy_df_clean['latitude'], errors='coerce')
# Filter out non-chicago locations
energy_df_clean = energy_df_clean[(energy_df_clean['latitude'].between(41.60, 42.10)) & (energy_df_clean['longitude'].between(-88.00, -87.50))]



energy_df_clean.head()

/var/folders/m6/k51w_3q94hn6j_yy3x_2g81m0000gn/T/ipykernel_60075/2359007054.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  energy_df_clean['data_year'] = pd.to_numeric(energy_df_clean['data_year'], errors='coerce')
/var/folders/m6/k51w_3q94hn6j_yy3x_2g81m0000gn/T/ipykernel_60075/2359007054.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  energy_df_clean['longitude'] = pd.to_numeric(energy_df_clean['longitude'], errors='coerce')
/var/folders/m6/k51w_3q94hn6j_yy3x_2g81m0000gn/T/ipykernel_60075/2359007

,data_year,id,reporting_status,address,zip_code,chicago_energy_rating,exempt_from_chicago_energy_rating,community_area,gross_floor_area_buildings_sq_ft,latitude,...,district_steam_use_kbtu,district_chilled_water_use_kbtu,site_eui_kbtu_sq_ft,source_eui_kbtu_sq_ft,weather_normalized_site_eui_kbtu_sq_ft,weather_normalized_source_eui_kbtu_sq_ft,total_ghg_emissions_metric_tons_co2e,ghg_intensity_kg_co2e_sq_ft,water_use_kgal,all_other_fuel_use_kbtu
3,2022,256888,Submitted Data,318 N CARPENTER ST,60607,4,False,NaN,104849,41.887365,...,0,0,32.4,61.9,32.8,61.4,311.5,3,NaN,NaN
18,2022,256882,Submitted Data,1648 W DIVISION ST,60622,1.5,False,NaN,106888,41.903424,...,0,0,105,138.1,108.6,139.8,638.8,7,NaN,NaN
24,2022,160184,Submitted Data,29 E Madison St,60602,2.5,False,LOOP,240076,41.881965,...,0,0,117,203.5,122.7,209.4,2361.5,9.8,NaN,NaN
33,2021,260184,Submitted,2728 N HAMPDEN CT,60614,1.5,False,LINCOLN PARK,170000,41.922011,...,0,0,117.6,161.6,115.5,159.4,1148.3,8.2,NaN,NaN
36,2021,260149,Submitted,3145 S Ashland Ave,60608,4,False,MCKINLEY PARK,74495,41.836579,...,0,0,176.5,373.7,176.5,373.7,1417.8,19,NaN,NaN


In [242]:
energy_df_clean.columns

Index(['data_year', 'id', 'reporting_status', 'address', 'zip_code',
       'chicago_energy_rating', 'exempt_from_chicago_energy_rating',
       'community_area', 'gross_floor_area_buildings_sq_ft', 'latitude',
       'longitude', 'location', 'row_id', ':@computed_region_43wa_7qmu',
       ':@computed_region_vrxf_vc4k', ':@computed_region_6mkv_f3dw',
       ':@computed_region_bdys_3d7i', ':@computed_region_awaf_s7ux',
       'property_name', 'primary_property_type', 'year_built', 'of_buildings',
       'energy_star_score', 'electricity_use_kbtu', 'natural_gas_use_kbtu',
       'district_steam_use_kbtu', 'district_chilled_water_use_kbtu',
       'site_eui_kbtu_sq_ft', 'source_eui_kbtu_sq_ft',
       'weather_normalized_site_eui_kbtu_sq_ft',
       'weather_normalized_source_eui_kbtu_sq_ft',
       'total_ghg_emissions_metric_tons_co2e', 'ghg_intensity_kg_co2e_sq_ft',
       'water_use_kgal', 'all_other_fuel_use_kbtu'],
      dtype='object')

In [293]:
energy_df_clean.head()
energy_df_clean = energy_df.dropna(subset=['data_year', 'longitude', 'latitude', 'energy_star_score'])
energy_df_clean['chicago_energy_rating'] = pd.to_numeric(energy_df_clean['chicago_energy_rating'], errors='coerce')


energy_grouped_comm = energy_df_clean.groupby('community_area').agg({
    "chicago_energy_rating": "sum",
    "address": "count",
}).reset_index()
energy_grouped_comm.columns = ["Community_Area", "Total_Energy_Rating", "Total_Buildings"]
energy_grouped_comm['Mean_Rating'] = energy_grouped_comm.apply(lambda x: x["Total_Energy_Rating"]/x["Total_Buildings"], axis=1)

display(energy_grouped_comm.sort_values("Total_Buildings"))

/var/folders/m6/k51w_3q94hn6j_yy3x_2g81m0000gn/T/ipykernel_60075/4265988090.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  energy_df_clean['chicago_energy_rating'] = pd.to_numeric(energy_df_clean['chicago_energy_rating'], errors='coerce')


,Community_Area,Total_Energy_Rating,Total_Buildings,Mean_Rating
155,"outside Chicago, in Bedford Park",3.0,1,3.000000
122,Riverdale,1.0,1,1.000000
50,Fuller Park,3.0,1,3.000000
90,Mckinley Park,3.5,1,3.500000
149,West Garfield Park,3.5,2,1.750000
...,...,...,...,...
102,Near North Side,1600.5,612,2.615196
73,LAKE VIEW,2243.0,1072,2.092351
96,NEAR WEST SIDE,2904.5,1410,2.059929
77,LOOP,3631.5,2046,1.774927


In [210]:
retrofit_df[['street_number','street_name']].head()
retrofit_df.columns

Index(['id', 'permit_', 'permit_status', 'permit_milestone', 'permit_type',
       'review_type', 'application_start_date', 'issue_date',
       'processing_time', 'street_number', 'street_direction', 'street_name',
       'work_description', 'building_fee_paid', 'zoning_fee_paid',
       'other_fee_paid', 'subtotal_paid', 'building_fee_unpaid',
       'zoning_fee_unpaid', 'other_fee_unpaid', 'subtotal_unpaid',
       'building_fee_waived', 'building_fee_subtotal', 'zoning_fee_subtotal',
       'other_fee_subtotal', 'zoning_fee_waived', 'other_fee_waived',
       'subtotal_waived', 'total_fee', 'reported_cost', 'pin_list',
       'community_area', 'census_tract', 'ward', 'xcoordinate', 'ycoordinate',
       'latitude', 'longitude', 'location'],
      dtype='object')

In [260]:
allhomes_df = result_vertical = pd.concat([retrofit_df, newbuild_df])
allhomes_df = allhomes_df[['id','permit_type','community_area','ward','latitude','longitude','location']]
allhomes_df.head()

,id,permit_type,community_area,ward,latitude,longitude,location
0,3316806,PERMIT - RENOVATION/ALTERATION,27,24,41.87153852754136,-87.71955488514301,"{'type': 'Point', 'coordinates': [-87.71955488..."
1,3362770,PERMIT - RENOVATION/ALTERATION,28,27,41.88338943530201,-87.66807949279023,"{'type': 'Point', 'coordinates': [-87.66807949..."
2,3368303,PERMIT - RENOVATION/ALTERATION,10,41,41.99913120190079,-87.7983587609477,"{'type': 'Point', 'coordinates': [-87.79835876..."
3,3373499,PERMIT - RENOVATION/ALTERATION,8,42,41.88787484815479,-87.63088485124779,"{'type': 'Point', 'coordinates': [-87.63088485..."
4,3378148,PERMIT - RENOVATION/ALTERATION,9,41,42.01519122083687,-87.806741706282,"{'type': 'Point', 'coordinates': [-87.80674170..."


In [177]:
""" WARD ZIPCODE ZONING CODE SETUP """
zipcodes_df = read_api("https://data.cityofchicago.org/resource/unjd-c2ca.json")
zipcodes_df.head()

200


,the_geom,objectid,zip,shape_area,shape_len
0,"{'type': 'MultiPolygon', 'coordinates': [[[[-8...",33,60647,106052287.488,42720.0444058
1,"{'type': 'MultiPolygon', 'coordinates': [[[[-8...",34,60639,127476050.762,48103.7827213
2,"{'type': 'MultiPolygon', 'coordinates': [[[[-8...",35,60707,45069038.4783,27288.6096123
3,"{'type': 'MultiPolygon', 'coordinates': [[[[-8...",36,60622,70853834.3797,42527.9896789
4,"{'type': 'MultiPolygon', 'coordinates': [[[[-8...",37,60651,99039621.2518,47970.1401531


In [299]:
""" WARD ZIPCODE ZONING CODE SETUP """
wards_df = read_api("https://data.cityofchicago.org/resource/p293-wvbd.json")
wards_df.head()

200


,ward,the_geom,objectid,edit_date,ward_id,globalid,st_area_sh,st_length_
0,1,"{'type': 'MultiPolygon', 'coordinates': [[[[-8...",51,2022-06-01T00:00:00.000,1,{DB2A2A7D-FAF1-42A4-B061-AE18C31A80BB},65893461.4401,61878.8215865
1,2,"{'type': 'MultiPolygon', 'coordinates': [[[[-8...",52,2022-06-01T00:00:00.000,2,{88F300F6-D6DF-4337-8DE3-0C2D27A5B338},31285111.9943,74175.9492394
2,3,"{'type': 'MultiPolygon', 'coordinates': [[[[-8...",53,2022-06-01T00:00:00.000,3,{4F169974-0F93-4C35-98C1-20A8415C72F5},97773414.0062,85625.5976201
3,4,"{'type': 'MultiPolygon', 'coordinates': [[[[-8...",54,2022-06-01T00:00:00.000,4,{02DC7BBB-BD1E-413C-994E-0A3154FD3D54},115809136.712,103594.867171
4,5,"{'type': 'MultiPolygon', 'coordinates': [[[[-8...",55,2022-06-01T00:00:00.000,5,{0A109A41-9DED-47D7-934E-1EA1CC7EE025},112080253.112,88207.6902413


In [287]:
communities_gdf = gpd.read_file('/Users/dinguid/PH Accelerator Dropbox/Dylan Ingui/Mac/Desktop/_Urban Technology/URP535 - Urban Informatics/URP535_Final/CommAreas_20250407.csv')
communities_gdf = communities_gdf[['the_geom','AREA_NUMBE','COMMUNITY','SHAPE_AREA']]
communities_gdf.head()

,the_geom,AREA_NUMBE,COMMUNITY,SHAPE_AREA
0,MULTIPOLYGON (((-87.60914087617894 41.84469250...,35,DOUGLAS,46004621.1581
1,MULTIPOLYGON (((-87.59215283879394 41.81692934...,36,OAKLAND,16913961.0408
2,MULTIPOLYGON (((-87.62879823733725 41.80189303...,37,FULLER PARK,19916704.8692
3,MULTIPOLYGON (((-87.6067081256125 41.816813770...,38,GRAND BOULEVARD,48492503.1554
4,MULTIPOLYGON (((-87.59215283879394 41.81692934...,39,KENWOOD,29071741.9283


In [258]:
import geopandas as gpd

# Step 1: Convert allhomes_df to a GeoDataFrame
allhomes_gdf = gpd.GeoDataFrame(
    allhomes_df,
    geometry=gpd.points_from_xy(allhomes_df['longitude'], allhomes_df['latitude']),
    crs="EPSG:4326"
)

# Step 2: Convert communities_df to a GeoDataFrame (skip wkt.loads step)
communities_gdf = gpd.GeoDataFrame(
    communities_gdf,
    geometry='the_geom',
    crs="EPSG:4326"
)

# Step 3: Spatial join to assign each home to a community
homes_with_community = gpd.sjoin(allhomes_gdf, communities_gdf, how='left', predicate='within')

# Step 4: Group by community area number and count permits
permit_counts = homes_with_community.groupby('AREA_NUMBE').size().reset_index(name='permit_count')

# Step 5: Merge permit counts into community_gdf
community_with_permits = communities_gdf.merge(permit_counts, on='AREA_NUMBE', how='left')
community_with_permits

,the_geom,AREA_NUMBE,COMMUNITY,SHAPE_AREA,permit_count
0,"MULTIPOLYGON (((-87.60914 41.84469, -87.60915 ...",35,DOUGLAS,46004621.1581,424
1,"MULTIPOLYGON (((-87.59215 41.81693, -87.59231 ...",36,OAKLAND,16913961.0408,189
2,"MULTIPOLYGON (((-87.6288 41.80189, -87.62879 4...",37,FULLER PARK,19916704.8692,111
3,"MULTIPOLYGON (((-87.60671 41.81681, -87.6067 4...",38,GRAND BOULEVARD,48492503.1554,1074
4,"MULTIPOLYGON (((-87.59215 41.81693, -87.59215 ...",39,KENWOOD,29071741.9283,347
...,...,...,...,...,...
72,"MULTIPOLYGON (((-87.69646 41.70714, -87.69644 ...",74,MOUNT GREENWOOD,75584290.0209,441
73,"MULTIPOLYGON (((-87.64215 41.68508, -87.64249 ...",75,MORGAN PARK,91877340.6988,358
74,"MULTIPOLYGON (((-87.83658 41.9864, -87.83658 4...",76,OHARE,371835607.687,244
75,"MULTIPOLYGON (((-87.65456 41.99817, -87.65456 ...",77,EDGEWATER,48449990.8397,1063


In [252]:
homes_with_community

,id,permit_type,community_area,ward,latitude,longitude,location,geometry,index_right,AREA_NUMBE,COMMUNITY,SHAPE_AREA
0,3316806,PERMIT - RENOVATION/ALTERATION,27,24,41.87153852754136,-87.71955488514301,"{'type': 'Point', 'coordinates': [-87.71955488...",POINT (-87.71955 41.87154),27.0,27,EAST GARFIELD PARK,53883220.8462
1,3362770,PERMIT - RENOVATION/ALTERATION,28,27,41.88338943530201,-87.66807949279023,"{'type': 'Point', 'coordinates': [-87.66807949...",POINT (-87.66808 41.88339),28.0,28,NEAR WEST SIDE,158492466.554
2,3368303,PERMIT - RENOVATION/ALTERATION,10,41,41.99913120190079,-87.7983587609477,"{'type': 'Point', 'coordinates': [-87.79835876...",POINT (-87.79836 41.99913),35.0,10,NORWOOD PARK,121959105.47
3,3373499,PERMIT - RENOVATION/ALTERATION,8,42,41.88787484815479,-87.63088485124779,"{'type': 'Point', 'coordinates': [-87.63088485...",POINT (-87.63088 41.88787),36.0,8,NEAR NORTH SIDE,76675895.9728
4,3378148,PERMIT - RENOVATION/ALTERATION,9,41,42.01519122083687,-87.806741706282,"{'type': 'Point', 'coordinates': [-87.80674170...",POINT (-87.80674 42.01519),76.0,9,EDISON PARK,31636313.7864
...,...,...,...,...,...,...,...,...,...,...,...,...
25249,3390382,PERMIT - NEW CONSTRUCTION,27,24,41.872655795152845,-87.70721421915886,"{'type': 'Point', 'coordinates': [-87.70721421...",POINT (-87.70721 41.87266),27.0,27,EAST GARFIELD PARK,53883220.8462
25250,3391773,PERMIT - NEW CONSTRUCTION,71,17,41.750729510516344,-87.64557103359442,"{'type': 'Point', 'coordinates': [-87.64557103...",POINT (-87.64557 41.75073),69.0,71,AUBURN GRESHAM,105065353.602
25251,3394840,PERMIT - NEW CONSTRUCTION,27,24,41.87265270488431,-87.70751585933291,"{'type': 'Point', 'coordinates': [-87.70751585...",POINT (-87.70752 41.87265),27.0,27,EAST GARFIELD PARK,53883220.8462
25252,3043824,PERMIT - NEW CONSTRUCTION,15,45,41.952510622379016,-87.75317109947974,"{'type': 'Point', 'coordinates': [-87.75317109...",POINT (-87.75317 41.95251),14.0,15,PORTAGE PARK,110196097.139


In [269]:
allhomes_df.head()

,id,permit_type,community_area,ward,latitude,longitude,location
0,3316806,PERMIT - RENOVATION/ALTERATION,27,24,41.87153852754136,-87.71955488514301,"{'type': 'Point', 'coordinates': [-87.71955488..."
1,3362770,PERMIT - RENOVATION/ALTERATION,28,27,41.88338943530201,-87.66807949279023,"{'type': 'Point', 'coordinates': [-87.66807949..."
2,3368303,PERMIT - RENOVATION/ALTERATION,10,41,41.99913120190079,-87.7983587609477,"{'type': 'Point', 'coordinates': [-87.79835876..."
3,3373499,PERMIT - RENOVATION/ALTERATION,8,42,41.88787484815479,-87.63088485124779,"{'type': 'Point', 'coordinates': [-87.63088485..."
4,3378148,PERMIT - RENOVATION/ALTERATION,9,41,42.01519122083687,-87.806741706282,"{'type': 'Point', 'coordinates': [-87.80674170..."


In [270]:
energy_df.head()

,data_year,id,reporting_status,address,zip_code,chicago_energy_rating,exempt_from_chicago_energy_rating,community_area,gross_floor_area_buildings_sq_ft,latitude,...,district_steam_use_kbtu,district_chilled_water_use_kbtu,site_eui_kbtu_sq_ft,source_eui_kbtu_sq_ft,weather_normalized_site_eui_kbtu_sq_ft,weather_normalized_source_eui_kbtu_sq_ft,total_ghg_emissions_metric_tons_co2e,ghg_intensity_kg_co2e_sq_ft,water_use_kgal,all_other_fuel_use_kbtu
0,2021,101671,Not Submitted,6101 6115 N SEELEY AVE,60659,0,False,WEST RIDGE,65250,41.99046172,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021,101826,Not Submitted,8136 8142 S DREXEL AVE,60619,0,False,CHATHAM,66888,41.74401302,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021,102323,Not Submitted,3515 3525 W FRANKLIN BLVD,60624,0,False,HUMBOLDT PARK,93000,41.8808242,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022,256888,Submitted Data,318 N CARPENTER ST,60607,4,False,NaN,104849,41.88736471232649,...,0,0,32.4,61.9,32.8,61.4,311.5,3,NaN,NaN
4,2021,104374,Not Submitted,6000 N SHERIDAN RD,60660,0,False,EDGEWATER,55686,41.99090057,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [274]:
energy_df_clean['latitude'] = energy_df_clean['latitude'].astype(float)
energy_df_clean['longitude'] = energy_df_clean['longitude'].astype(float)

allhomes_df.loc[:, 'latitude'] = allhomes_df['latitude'].astype(float)
allhomes_df.loc[:, 'longitude'] = allhomes_df['longitude'].astype(float)
allhomes_df = allhomes_df.copy()

merged = energy_df_clean.merge(allhomes_df, on=['latitude', 'longitude'])

In [276]:
merged.shape, merged.columns

((7604, 40),
 Index(['data_year', 'id_x', 'reporting_status', 'address', 'zip_code',
        'chicago_energy_rating', 'exempt_from_chicago_energy_rating',
        'community_area_x', 'gross_floor_area_buildings_sq_ft', 'latitude',
        'longitude', 'location_x', 'row_id', ':@computed_region_43wa_7qmu',
        ':@computed_region_vrxf_vc4k', ':@computed_region_6mkv_f3dw',
        ':@computed_region_bdys_3d7i', ':@computed_region_awaf_s7ux',
        'property_name', 'primary_property_type', 'year_built', 'of_buildings',
        'energy_star_score', 'electricity_use_kbtu', 'natural_gas_use_kbtu',
        'district_steam_use_kbtu', 'district_chilled_water_use_kbtu',
        'site_eui_kbtu_sq_ft', 'source_eui_kbtu_sq_ft',
        'weather_normalized_site_eui_kbtu_sq_ft',
        'weather_normalized_source_eui_kbtu_sq_ft',
        'total_ghg_emissions_metric_tons_co2e', 'ghg_intensity_kg_co2e_sq_ft',
        'water_use_kgal', 'all_other_fuel_use_kbtu', 'id_y', 'permit_type',
        'comm

In [285]:
# Round to 4 or 5 decimal places (approx ~11m or ~1m accuracy)
energy_df_clean['lat_round'] = energy_df_clean['latitude'].round(5)
energy_df_clean['lon_round'] = energy_df_clean['longitude'].round(5)

allhomes_df['lat_round'] = allhomes_df['latitude'].round(5)
allhomes_df['lon_round'] = allhomes_df['longitude'].round(5)

merged = energy_df_clean.merge(allhomes_df, on=['lat_round', 'lon_round'])
df = merged[['data_year', 'id_x',  'zip_code',  'chicago_energy_rating', 'latitude_x', 'longitude_x', 'location_x', 'property_name', 'year_built', 'community_area_y', 'permit_type', 'ward']]
df.columns = ['data_year', 'id',  'zip_code',  'chicago_energy_rating', 'latitude', 'longitude', 'location', 'property_name', 'year_built', 'community_area', 'permit_type', 'ward']
display(df)

,data_year,id,zip_code,chicago_energy_rating,latitude,longitude,location,property_name,year_built,community_area,permit_type,ward
0,2022,256888,60607,4,41.887365,-87.653505,"{'latitude': '41.88736471232649', 'longitude':...",NaN,2019,28,PERMIT - RENOVATION/ALTERATION,27
1,2022,256888,60607,4,41.887365,-87.653505,"{'latitude': '41.88736471232649', 'longitude':...",NaN,2019,28,PERMIT - RENOVATION/ALTERATION,27
2,2022,256888,60607,4,41.887365,-87.653505,"{'latitude': '41.88736471232649', 'longitude':...",NaN,2019,28,PERMIT - NEW CONSTRUCTION,27
3,2022,256888,60607,4,41.887365,-87.653505,"{'latitude': '41.88736471232649', 'longitude':...",NaN,2019,28,PERMIT - NEW CONSTRUCTION,27
4,2022,256888,60607,4,41.887365,-87.653505,"{'latitude': '41.88736471232649', 'longitude':...",NaN,2019,28,PERMIT - NEW CONSTRUCTION,27
...,...,...,...,...,...,...,...,...,...,...,...,...
16179,2023,256671,60608,4,41.789841,-87.622490,"{'latitude': '41.789841', 'longitude': '-87.62...",Chantico Lofts,2007,40,PERMIT - RENOVATION/ALTERATION,20
16180,2023,251300,60614,2,41.908254,-87.639863,"{'latitude': '41.908254', 'longitude': '-87.63...",2020 Lincoln Park West Condominium Association,1972,8,PERMIT - RENOVATION/ALTERATION,27
16181,2023,106052,60660,4,41.773447,-87.568490,"{'latitude': '41.773447', 'longitude': '-87.56...","6214 N. Winthrop, LLC",1969,43,PERMIT - RENOVATION/ALTERATION,5
16182,2023,106052,60660,4,41.773447,-87.568490,"{'latitude': '41.773447', 'longitude': '-87.56...","6214 N. Winthrop, LLC",1969,43,PERMIT - RENOVATION/ALTERATION,5
